In [1]:
import numpy as np
import scipy as sp
import pandas as pd

In [2]:
from collections import deque
def bfs(graph,start):
  visited = set()
  queue = deque([start])
  result = []
  while queue:
    vertex = queue.popleft()
    if veretex not  in visited:
      visited.add(vertex)
      result.append(vertex)

In [3]:
def bfs(graph, start_node):
    visited = set()
    queue = [start_node]
    result = []
    while queue:
        vertex = queue.pop(0)
        if vertex not in visited:
            result.append(vertex)
            visited.add(vertex)
            queue.extend(neighbour for neighbour in graph[vertex] if neighbour not in visited)
    return result
if __name__ == '__main__':
    graph = {
        '0': ['1', '3'],
        '1': ['0', '2', '3'],
        '3': ['0', '4'],
        '2': ['1'],
        '4': ['2', '5'],
        '5': ['2', '4']
    }
    start_node = '0'
    print(f"BFS traversal starting from vertex {start_node}: {bfs(graph, start_node)}")

BFS traversal starting from vertex 0: ['0', '1', '3', '2', '4', '5']


In [4]:

graph1={
    'A':['B','C','G'],
    'B':['A'],
    'C':['A','D','F'],
    'D':['A','C','E','H'],
    'E':['D','H','F'],
    'F':['C','E'],
    'G':['A','D','H'],
    'H':['D','E','G']
}

def dfs(graph,node,visited):
  if node not in visited:
    visited.append(node)
    for k in graph[node]:
      dfs(graph,k,visited)
  return visited

visited=dfs(graph1,'B',[])
print(visited)

['B', 'A', 'C', 'D', 'E', 'H', 'G', 'F']


In [5]:
from copy import deepcopy
import numpy as np
import time

def bestsolution(state):
    bestsol = np.array([], int).reshape(-1, 9)
    count = len(state) - 1
    while count != -1:
        bestsol = np.insert(bestsol, 0, state[count]['puzzle'], 0)
        count = state[count]['parent']
    return bestsol.reshape(-1, 3, 3)

def misplaced_tiles(puzzle, goal):
    mscost = np.sum(puzzle != goal) - 1
    return mscost if mscost > 0 else 0

def coordinates(puzzle):
    pos = np.array(range(9))
    for p, q in enumerate(puzzle):
        pos[q] = p
    return pos

def evaluvate_misplaced(puzzle, goal):
    steps = np.array([
        ('up', [0, 1, 2], -3),
        ('down', [6, 7, 8], 3),
        ('left', [0, 3, 6], -1),
        ('right', [2, 5, 8], 1)
    ], dtype=[('move', str, 1), ('position', list), ('head', int)])

    dtstate = np.dtype([
        ('puzzle', list),
        ('parent', int),
        ('gn', int),
        ('hn', int)
    ])

    costg = coordinates(goal)
    parent = -1
    gn = 0
    hn = misplaced_tiles(coordinates(puzzle), costg)
    state = np.array([(puzzle, parent, gn, hn)], dtstate)

    dtpriority = np.dtype([('position', int), ('fn', int)])
    priority = np.array([(0, hn)], dtpriority)

    while True:
        priority = np.sort(priority, kind='mergesort', order=['fn', 'position'])
        position, fn = priority[0]
        priority = np.delete(priority, 0, 0)
        puzzle, parent, gn, hn = state[position]
        puzzle = np.array(puzzle)

        blank = int(np.where(puzzle == 0)[0][0])
        gn += 1
        start_time = time.time()

        for s in steps:
            if blank not in s['position']:
                openstates = deepcopy(puzzle)
                openstates[blank], openstates[blank + s['head']] = openstates[blank + s['head']], openstates[blank]

                if not (np.all(list(state['puzzle']) == openstates, 1)).any():
                    end_time = time.time()
                    if (end_time - start_time) > 2:
                        print("The 8 puzzle is unsolvable\n")
                        break

                    hn = misplaced_tiles(coordinates(openstates), costg)
                    q = np.array([(openstates, position, gn, hn)], dtstate)
                    state = np.append(state, q, 0)
                    fn = gn + hn
                    q = np.array([(len(state) - 1, fn)], dtpriority)
                    priority = np.append(priority, q, 0)

                    if np.array_equal(openstates, goal):
                        print('The 8 puzzle is solvable\n')
                        return state, len(priority)

    return state, len(priority)

puzzle = [2, 8, 3, 1, 6, 4, 7, 0, 5]
goal = [1, 2, 3, 8, 0, 4, 7, 6, 5]

state, visited = evaluvate_misplaced(puzzle, goal)
bestpath = bestsolution(state)
print(str(bestpath).replace('[', ' ').replace(']', ''))
totalmoves = len(bestpath) - 1
print('\nSteps to reach goal:', totalmoves)
visit = len(state) - visited
print('Total nodes visited: ', visit, "\n")

The 8 puzzle is solvable

   2 8 3
   1 6 4
   7 0 5

   2 8 3
   1 0 4
   7 6 5

   2 0 3
   1 8 4
   7 6 5

   0 2 3
   1 8 4
   7 6 5

   1 2 3
   0 8 4
   7 6 5

   1 2 3
   8 0 4
   7 6 5

Steps to reach goal: 5
Total nodes visited:  6 

